In [28]:
import cv2
import numpy as np
import time 	


def process(path):
     # 根据路径读取一张图片
    start=time.clock()    
    image = cv2.imread(path)
    shapes=image.shape
    print('shapes',shapes)
    # 剪切有效区域 ：判断图片的大小，是否需要裁剪
    images_size=[shapes[0],shapes[1]]
    images_size=np.min(images_size)
    if images_size > 1000:
        image=image[0:int(1/3*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    elif images_size > 800:
        image=image[int(1/6*shapes[1]):int(5/6*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    else:
        image=image
#     cv2.imshow('image',image)
    # 计算有效区域的面积
    shapes=image.shape
    propotion=image.shape[0]*image.shape[1]
     # BGR转HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #保留底片
    origin_copy=image.copy()
    return hsv,propotion,shapes,origin_copy


# 绿色的范围
def getGreen(image):    
    lower_green = np.array([45, 70, 100])
    upper_green = np.array([85, 255, 255])
    return {'green':getRange(image,lower_green,upper_green)}

# 红色的范围
def getRed(image):    
    low_red = np.array([0, 100, 80])
    high_red = np.array([8, 255, 255])
    return {'red':getRange(image,low_red,high_red)}

# 绿色的范围
def getYellow(image):    
    low_yellow = np.array([10, 100, 220])
    high_yellow = np.array([45, 255, 255])
    return {'yellow':getRange(image,low_yellow,high_yellow)}

def getRange(image,low,high):
# cv2.inRange函数设阈值，去除背景部分
    mask = cv2.inRange(image, low, high)
    return mask

def deNoise(img):   
    erode=cv2.erode(img,(100,100),iterations=4)
#降噪（模糊处理用来减少瑕疵点）
    blur = cv2.blur(erode, (5,5))   
# 膨胀
    dilate=cv2.dilate(blur,(100,100),iterations=4)
# 边缘检测    
    canny = cv2.Canny(dilate, 150, 240)     
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
# 闭操作    
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
#     cv2.imshow('closed',closed)
    return closed

#非最大值抑制,减少重叠区域
def non_max_suppression_slow(boxes, overlapThresh):  
    if len(boxes) == 0:  
        return []  
   
    pick = []  
    x1 = boxes[:,0]  
    y1 = boxes[:,1]  
    x2 = boxes[:,2]  
    y2 = boxes[:,3]  
   
    area = (x2 - x1 + 1) * (y2 - y1 + 1)  
    idxs = np.argsort(y2)
    while len(idxs) > 0:  
        last = len(idxs) - 1  
        i = idxs[last]  
        pick.append(i)  
        suppress = [last]  
        for pos in range(0, last):  
            j = idxs[pos]  
            xx1 = max(x1[i], x1[j])  
            yy1 = max(y1[i], y1[j])  
            xx2 = min(x2[i], x2[j])  
            yy2 = min(y2[i], y2[j])  

            w = max(0, xx2 - xx1 + 1)  
            h = max(0, yy2 - yy1 + 1)  

            overlap = float(w * h) / area[j]  
            print('overlap',overlap,'overlapThresh',overlapThresh)
            if overlap > overlapThresh:  
                suppress.append(pos)  
        idxs = np.delete(idxs, suppress)  
    return boxes[pick]  
    
#获取最终ROI的坐标    
def getPositions(img,propotion,shapes,origin_copy):
    ret,thresh = cv2.threshold(img,127,255,0)
    tmp,contours,hierarchy=cv2.findContours(thresh,1,2)
    
#     print(tmp,'tmp','contours',contours,len(contours),'hierarchy',hierarchy,hierarchy)
    print('contours:',len(contours))        
    area_list = []
    order=0
#有效交通灯的比例    
    if contours :
        for i in range(len(contours)):
            if cv2.contourArea(contours[i])/propotion >0.0002 and cv2.contourArea(contours[i])/propotion < 0.1:
# 最小面积矩形                
                x,y,w,h=cv2.boundingRect(contours[i])
#                 cv2.rectangle(origin_copy,(x,y),(x+w,y+h),(0,255,0),2)
                y0=int(y-0.2*h)
                y1=int(y+0.9*h)
                x0=int(x-0.1*w)
                x1=int(x+w)
                if y0 <0:
                    y0=0
                if y1>shapes[0]:
                    y1=shapes[0]
                if x0<0:
                    x0=0
                if x1>shapes[1]:
                    x1=shapes[1]
                area_list.append([x0,y0,x1,y1])
        area_list=np.array(area_list)
        prop=non_max_suppression_slow(area_list,0.7)
        print('prop',prop)
        if prop.any():
            for (startX, startY, endX, endY) in prop:  
                cv2.rectangle(origin_copy, (startX, startY), (endX, endY), (0, 255, 0), 2) 
            order=int(np.floor((len(prop))/2))
            print(order)
            target=prop[order]
            cv2.imshow('res',origin_copy)
            return [target[1],target[3],target[0],target[2]]
        else:
            return 0
    else:
        return 0
#                 area_list.append((i,cv2.contourArea(contours[i])))
# # 画最小外接矩形
#                 rect=cv2.minAreaRect(contours[i])
#                 box=np.int0(cv2.boxPoints(rect))        
# # #用红色在原图画出所有ROI区域
#                 cv2.drawContours(origin_copy,[box],-1,(0,0,255),2)  

# # 所有的ROI区域就是按从大到小开始排序的
#         print('area_list',area_list)
# # 求所有ROI区域中的中位数    
#         order=int(np.ceil((len(area_list))/2))
#         print('order:',order)
#         if order>0:
#             cnt=contours[order] 
            
# #按最小面积计算 HSV的坐标    
#             x,y,w,h=cv2.boundingRect(cnt)
# #             cv2.rectangle(origin_copy,(x,y),(x+w,y+h),(0,255,0),2)
#             y0=int(y-h)
#             y1=int(y+h)
#             x0=int(x-w)
#             x1=int(x+w)
#             if y0 <0:
#                 y0=0
#             if y1>shapes[0]:
#                 y1=shapes[0]
#             if x0<0:
#                 x0=0
#             if x1>shapes[1]:
#                 x1=shapes[1]
#             cv2.imshow('res',origin_copy)
#             return [y0,y1,x0,x1]
#         else:
#             return 0
#     else:
#         return 0
    



def main(path):
    start=time.clock()
    hsv,propotion,shapes,origin_copy=process(path)    
    colors=[]
    colors.append(getGreen(hsv))
    colors.append(getRed(hsv))
    colors.append(getYellow(hsv))
    # 节省时间复杂度

    areas=[]
    sumResult=0
    _max={'color':0}
    for i in range(len(colors)):
        closed = deNoise(list(colors[i].values())[0])
        coordinate = getPositions(closed,propotion,shapes,origin_copy)
        if isinstance(coordinate,list):
            print('coordinate',coordinate)
            HSV=origin_copy[coordinate[0]:coordinate[1],coordinate[2]:coordinate[3],:]
            HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
            H, S, V = cv2.split(HSV)
            v = np.mean(V)
            areas.append((list(colors[i].keys())[0],v))
        else:
            areas.append((list(colors[i].keys())[0],0))

    # 按颜色值的大小排序
    areas=sorted(areas,key=lambda x : (x[1]),reverse=True)
    print(areas)
    print(_max)
    ended=time.clock()
    print('total:',ended-start)
    if int(areas[0][1]) != 0:
        print('The current color is:',areas[0][0])
    else:
        print('没有交通灯')
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__=='__main__':
    path="light3.jpg"
    main(path)

shapes (517, 519, 3)
contours: 2
overlap 1.0 overlapThresh 0.7
prop [[212  76 303 142]]
0
coordinate [76, 142, 212, 303]
contours: 14
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.00644329896907 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 overlapThresh 0.7
overlap 0.0 o